In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras import models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import PIL.Image as img
from keras.layers import Dense, Conv2D , MaxPool2D , Dropout , Flatten
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
data = pd.read_csv('/kaggle/input/skin-cancer-mnist-ham10000/hmnist_28_28_L.csv')
data_rescale = data/255
data_rescale

In [ ]:
data.describe()

In [ ]:
data.groupby('label').count()['pixel0001']

In [ ]:
data.isnull().sum()

In [ ]:
plt.figure(figsize=(8,8))
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(data_rescale.iloc[i,:784].values.reshape(28,28))
    plt.title(data.iloc[i,784])

In [ ]:
x = data_rescale.iloc[:,0:784]
y = data_rescale['label']*255

In [ ]:
EarlyStop = keras.callbacks.EarlyStopping(
               min_delta = 0.001,
               patience = 10 ,
               verbose = 0
)
model = keras.Sequential([
        preprocessing.RandomFlip('horizontal'),
        layers.Conv2D(20 , kernel_size =9),
        layers.MaxPool2D(pool_size=1),
        layers.Conv2D(16, kernel_size = 7),
        layers.MaxPool2D(pool_size=2),
        layers.Flatten(),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(7, activation = 'relu'),
        layers.Dense(7,activation='softmax')
]
)
model.compile(
    optimizer ='adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']

In [ ]:
channel = np.zeros(784).transpose()
x3d = np.stack((x.iloc[0,:],channel,channel),axis=-1)
x3d1 = x3d.reshape(28,28,3)
plt.subplot(1,2,1)
plt.imshow(data_rescale.iloc[0,:784].values.reshape(28,28))
plt.title('Grey Scale')
plt.subplot(1,2,2)
plt.imshow(x3d1)
plt.title('RGB')

In [ ]:
x_new = x.values.reshape(10015,28,28,1)
x_train , x_test , y_train ,y_test = train_test_split(x_new,y,test_size=0.2)
print(x_train.shape,
x_test.shape)
y_train1 = to_categorical(y_train, num_classes=7)
y_test1 = to_categorical(y_test, num_classes=7)

In [ ]:
train = model.fit(x_train,
          y_train1,
          batch_size = 1024,
          epochs= 100,
          callbacks=[EarlyStop],
          validation_data=(x_test,y_test1) )
history_df = pd.DataFrame(train.history)
history_df.loc[:, ['loss', 'val_loss']].plot();

In [ ]:
pred = model.predict(x_test)
pred1 = np.argmax(pred,axis=1)
pred1[0:100]